<a href="https://colab.research.google.com/github/poliakovfm/itmo_transcriptomics_homeworks/blob/main/HW0_Trans_Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from Bio.Seq import Seq
import warnings
warnings.simplefilter('ignore')

In [98]:
# function that finds all ORFs in a given sequence
# problem inspired by https://rosalind.info/problems/orf/

def find_all_orfs():

  # принимаем строку на вход
  input_sequence = input()

  # создаем дф со строками в разных направлениях
  sequences = pd.DataFrame({
      'forward': [input_sequence],
      'reverse': [input_sequence[::-1]],
      'complement': [str(Seq(input_sequence).complement())],
      'reverse complement': [str(Seq(input_sequence).reverse_complement())]
  })

  # создаем дф, куда будем записывать результаты
  orf_df = pd.DataFrame(columns=['orf_nt_sequence', 'orf_aa_sequence', 'start_position', 'end_position', 'nt_length', 'aa_length', 'direction'])

  # задаем старт и стоп кодоны
  start_codon = 'ATG'
  stop_codons = ['TAA', 'TAG', 'TGA']


  #обходим все направления строки
  for i, sequence in enumerate(list(sequences.iloc[0])):

    # создаем два пустых списка, куда мы будем записывать координаты начала соответствущих кодонов
    start_codons_positions = []
    stop_codons_positions = []

    # заполняем эти списки
    start_codons_positions = [j for j in range(len(sequence)) if sequence.startswith(start_codon, j)]

    for stop_codon in stop_codons:
      single_codon_positions = [j for j in range(len(sequence)) if sequence.startswith(stop_codon, j)]
      stop_codons_positions += single_codon_positions


    for start_codon_position in start_codons_positions:
      for stop_codon_position in stop_codons_positions:

        # 1 условие: координата старт кодона лежит раньше координаты стоп кодона
        if (start_codon_position < stop_codon_position):
          stop_codons_positions_updated = [k for k in stop_codons_positions if k % 3 == stop_codon_position % 3]

          if (stop_codon_position == min([k for k in stop_codons_positions_updated if k > start_codon_position])
            and (start_codon_position + 3) % 3 == stop_codon_position % 3):


            # делаем срез строки по соответствующим координатам
            orf_sequence =  sequence[start_codon_position:stop_codon_position]

            # заполняем наш дф соответствующими значениями
            orf_df = orf_df.append(pd.Series(), ignore_index=True)
            orf_df.iloc[len(orf_df.index)-1].loc['orf_nt_sequence'] = orf_sequence
            orf_df.iloc[len(orf_df.index)-1].loc['orf_aa_sequence'] = str(Seq(orf_sequence).translate())
            orf_df.iloc[len(orf_df.index)-1].loc['start_position'] = start_codon_position
            orf_df.iloc[len(orf_df.index)-1].loc['end_position'] = stop_codon_position
            orf_df.iloc[len(orf_df.index)-1].loc['nt_length'] = len(orf_sequence)
            orf_df.iloc[len(orf_df.index)-1].loc['aa_length'] = len(str(Seq(orf_sequence).translate()))
            orf_df.iloc[len(orf_df.index)-1].loc['direction'] = sequences.columns[i]




  return orf_df


In [99]:
# running the function and saving its output to new_df
# test input string: "AGCCTTGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"

new_df = find_all_orfs()

AGCCTTGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG


In [100]:
# checking its contents

new_df

,orf_nt_sequence,orf_aa_sequence,start_position,end_position,nt_length,aa_length,direction
0,ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAA,MGMTPRLGLESLLE,24,66,42,14,forward
1,ATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAA,MTPRLGLESLLE,30,66,36,12,forward
2,ATGTACCCCTACTGGGGCGCTGAACCTAATCTCAGAGAAAACCTTA...,MYPYWGAEPNLRENLIRTY,21,78,57,19,complement
3,ATGCTACTCGGATCATTCAGGCTTATTCCAAAAGAGACTCTAATCC...,MLLGSFRLIPKETLIQVAGSSPCNLS,5,83,78,26,reverse complement
4,ATG,M,70,73,3,1,reverse complement
